In [ ]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 24.6 MB/s eta 0:00:00


In [ ]:
import telebot
import pandas as pd
from telebot.types import ReplyKeyboardMarkup, KeyboardButton

ModuleNotFoundError: No module named 'telebot'

In [ ]:

#  API токен
TOKEN = "7757301495:AAHWGNrUXu_3J_JjSeje4W6TI0pZc4PxqwY"
bot = telebot.TeleBot(TOKEN)


In [ ]:
# Загружаем датасет
file_path = '/content/sample_data/PushkinLab2025.csv'  # Загруженный CSV-файл
df = pd.read_csv(file_path, names=['question', 'answer'], dtype=str)  # Переименовываем столбцы

In [ ]:
import pandas as pd
df = pd.read_csv('/content/sample_data/PushkinLab2025.csv', names=['question', 'answer'], dtype=str)
print(df.head(10))  # Покажет первые 10 строк


In [ ]:
# Проверяем, загружены ли данные
if df.empty:
    print("⚠️ Ошибка: CSV-файл пустой или не найден!")
else:
    print(f"✅ Успешно загружено {len(df)} записей.")

✅ Успешно загружено 1018 записей.


In [ ]:
def get_answer(question):
    question = question.strip().lower()  # Приводим текст к единому формату
    df['question'] = df['question'].astype(str).str.strip().str.lower()  # Приводим вопросы в таблице
    answer_row = df[df['question'] == question]

    if not answer_row.empty:
        return answer_row.iloc[0]['answer']  # Получаем первый найденный ответ
    else:
        return "Я пока не знаю ответа на этот вопрос. Попробуй выбрать один из примеров ниже!"


In [ ]:
# Функция генерации клавиатуры
def generate_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    for question in example_questions:
        markup.add(KeyboardButton(f"🎨 {question}"))
    markup.add(KeyboardButton("🔄 Обновить вопросы"))
    return markup


In [ ]:
# Получаем 5 примеров вопросов из датасета
num_questions = min(5, len(df['question'].dropna()))  # Берем минимум из 5 или всех доступных
example_questions = df['question'].dropna().sample(num_questions).tolist()

In [ ]:
def generate_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    for question in example_questions:
        markup.add(KeyboardButton(f"🎨 {question}"))
    markup.add(KeyboardButton("🔄 Обновить вопросы"))
    return markup

In [ ]:
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(
        message.chat.id,
        "🎨 Привет! Я чат-бот Пушкинского Музея **PushkinLab**.\n"
        "Я расскажу тебе про картины и художников Пушкинского музея. Спрашивай меня об **импрессионистах, постимпрессионистах, модернистах** и их произведениях!\n\n"
        "Попробуй задать один из этих вопросов или напиши свой:\n\n"
        + "\n".join([f"👉 {q}" for q in example_questions]),
        reply_markup=generate_keyboard()
    )

In [ ]:

@bot.message_handler(func=lambda message: message.text == "🔄 Обновить вопросы")
def refresh_questions(message):
    global example_questions
    num_questions = min(5, len(df['question'].dropna()))
    example_questions = df['question'].dropna().sample(num_questions).tolist()
    bot.send_message(
        message.chat.id,
        "🔄 Обновил список вопросов! Попробуй выбрать из новых примеров или задай свой вопрос.\n\n"
        + "\n".join([f"👉 {q}" for q in example_questions]),
        reply_markup=generate_keyboard()
    )

In [ ]:

@bot.message_handler(func=lambda message: True)
def reply_to_message(message):
    user_question = message.text.strip()
    answer = get_answer(user_question)
    bot.send_message(message.chat.id, f"✍️ {answer}", reply_markup=generate_keyboard())

In [ ]:

# Запуск бота
if __name__ == "__main__":
    print("Бот запущен!")
    bot.polling(none_stop=True)


Бот запущен!
